# Part II: Calibration

## Introduction

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn

sns.set_style('whitegrid')
colormap = sns.color_palette("Blues", as_cmap=True)

%matplotlib inline

import warnings 
warnings.filterwarnings('ignore')

The units of the columns are the following:

- Temperature: °$C$
- Humidity(Relative): $\%$
- Ozone: $K\Omega$ 
- Target(Ozone): $\mu gr/m^3$

In [ ]:
df = pd.read_csv('data.csv', sep=';')

In [ ]:
df.head()

In [ ]:
print(f'Columns: {df.shape[1]}\nRows: {df.shape[0]}')
print('Columns types:')
print(df.dtypes)

Fix date type and sort by date:

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df['date'][:5]

Fix columns' name and order:

In [ ]:
new_columns = {
    'date': 'date',
    'RefSt': 'target',
    'Sensor_O3': 'ozone',
    'Temp': 'temperature',
    'RelHum': 'humidity'
}
df.rename(columns=new_columns, inplace=True)
new_order = ['date', 'temperature', 'humidity', 'ozone', 'target']
df = df.reindex(columns=new_order)
df.columns

In [ ]:
df.describe().loc[['mean','std']]